In [3]:
from collections import namedtuple
import pandas as pd

In [4]:
df = pd.DataFrame(
    {
        'start_delivery_code':['a','a','e','e','a','a'],
        'touch_delivery_code':['a','b','e','g','a','h'],
        'landing_delivery_code':['b','c','g','f','h','g'],
        'delivery_group':['a,b,c','a,b,c','e,f,g','e,f,g','a,h,g','a,h,g'],
        'touch_exposure':[1,1,1,1,1,1],
        'landing_exposure':[1,1,1,1,1,1],
        'touch_click':[1,1,1,1,1,1],
        'landing_click':[1,1,1,1,1,1]
    }
)
df
# df.columns.to_list()

,start_delivery_code,touch_delivery_code,landing_delivery_code,delivery_group,touch_exposure,landing_exposure,touch_click,landing_click
0,a,a,b,"a,b,c",1,1,1,1
1,a,b,c,"a,b,c",1,1,1,1
2,e,e,g,"e,f,g",1,1,1,1
3,e,g,f,"e,f,g",1,1,1,1
4,a,a,h,"a,h,g",1,1,1,1
5,a,h,g,"a,h,g",1,1,1,1


In [5]:
landtuple = namedtuple('landtuple',df.columns.to_list()) 
#['start_delivery_code', 
# 'touch_delivery_code', 
# 'landing_delivery_code', 
# 'delivery_group', 
# 'touch_exposure',
# 'landing_exposure',
# 'touch_click',
# 'landing_click']
tgt_columns=['delivery_code','exposure','click','delivery_group','delivery_group_name','rnk']
sorted_chain = []
for name, group in df.groupby('delivery_group'):
    chain_dict = {}
    for i, row in group.iterrows():
        landtplInfo = landtuple(*row)
        chain_dict[landtplInfo.touch_delivery_code] = landtplInfo
    
    start = group.reset_index().loc[0,'start_delivery_code']   
    cur_touch = start
    delivery_group_name = []
    rnk = 1
    while cur_touch:
        land = chain_dict.get(cur_touch)
        if land:
            sorted_chain.append([land.touch_delivery_code + '(touch)',land.touch_exposure,land.touch_click,land.delivery_group,'',rnk])
            sorted_chain.append([land.landing_delivery_code + '(landing)',land.landing_exposure,land.landing_click,land.delivery_group,'',rnk+1])
            cur_touch = land.landing_delivery_code
            rnk += 2
            delivery_group_name
        else:
            break

# added delivery group name
df_target = pd.DataFrame(sorted_chain,columns=tgt_columns)
for name, group in df_target.groupby('delivery_group'):
    df_target.loc[df_target['delivery_group'] == name,'delivery_group_name'] = '->'.join(group['delivery_code'])
df_target


,delivery_code,exposure,click,delivery_group,delivery_group_name,rnk
0,a(touch),1,1,"a,b,c",a(touch)->b(landing)->b(touch)->c(landing),1
1,b(landing),1,1,"a,b,c",a(touch)->b(landing)->b(touch)->c(landing),2
2,b(touch),1,1,"a,b,c",a(touch)->b(landing)->b(touch)->c(landing),3
3,c(landing),1,1,"a,b,c",a(touch)->b(landing)->b(touch)->c(landing),4
4,a(touch),1,1,"a,h,g",a(touch)->h(landing)->h(touch)->g(landing),1
5,h(landing),1,1,"a,h,g",a(touch)->h(landing)->h(touch)->g(landing),2
6,h(touch),1,1,"a,h,g",a(touch)->h(landing)->h(touch)->g(landing),3
7,g(landing),1,1,"a,h,g",a(touch)->h(landing)->h(touch)->g(landing),4
8,e(touch),1,1,"e,f,g",e(touch)->g(landing)->g(touch)->f(landing),1
9,g(landing),1,1,"e,f,g",e(touch)->g(landing)->g(touch)->f(landing),2
